In [ ]:
import os
import math
from collections import namedtuple

import requests
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets

from tqdm.auto import tqdm
from IPython.display import display

tqdm.pandas()

In [ ]:
API_HOST = os.getenv('API_HOST') or 'http://localhost:8000'

In [ ]:
def clear_fields(fields):
    for f in fields:
        if isinstance(f, widgets.Text):
            f.value = ''
        elif isinstance(f, widgets.IntSlider) or isinstance(f, widgets.BoundedIntText):
            f.value = f.min
        elif isinstance(f, widgets.IntText):
            f.value = 0
        elif isinstance(f, widgets.Checkbox):
            f.value = False
        elif not isinstance(f, widgets.Dropdown):
            f.value = None


def clear_button(im):
    button = widgets.Button(description='Limpar campos')
    button.on_click(lambda b: clear_fields(im.widget.children))
    return button


def change_field_description(fields, old_value, new_value):
    for w in fields:
        if hasattr(w, 'description') and w.description == old_value:
            w.description = new_value


def add_accordion_tab(title, description, output, on_click_function, **kwargs):
    form = widgets.Output()
    with form:
        if description is not None:
            accordion_description = widgets.Accordion(
                children=[widgets.HTML(value=description)], selected_index=None
            )
            accordion_description.set_title(0, 'Descrição')
            display(accordion_description)
        im = widgets.interact_manual(on_click_function, **kwargs)
        display(clear_button(im))
        change_field_description(im.widget.children, 'Run Interact', 'Pesquisar')
    accordion_children.append(widgets.VBox([form, output]))
    accordion_titles.append(title)


accordion_children = []
accordion_titles = []

In [ ]:
anos = requests.get(
    f'{API_HOST}/api/f_comex/column-values?column=ANO'
).json()
movimentacoes = requests.get(
    f'{API_HOST}/api/f_comex/column-values?column=MOVIMENTACAO'
).json()
produtos = requests.get(
    f'{API_HOST}/api/d_sh2/column-values?column=NO_SH2_POR'
).json()

In [ ]:
months = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

states = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

In [ ]:
def get_comex(ano=None, movimentacao=None, produto=None, page=1, size=50):
    resp = requests.get(
        f'{API_HOST}/api/f_comex/',
        params={
            'ano': ano,
            'movimentacao': movimentacao,
            'no_sh2_por': produto,
            'page': page,
            'size': size,
        }
    )
    return resp.json()

In [ ]:
def fetch_data(ano=None, movimentacao=None, produto=None):
    comex_data = []
    size = 100
    data = get_comex(
        ano=ano, movimentacao=movimentacao, produto=produto,
        page=1, size=size
    )
    total = data.get('total')
    size = data.get('size')
    n_requests = math.ceil(total / size)

    for p in tqdm(range(2, n_requests + 3), desc='Requesting data'):
        items = data.get('items')
        if not items:
            break
        comex_data.extend(items)
        data = get_comex(
            ano=ano, movimentacao=movimentacao, produto=produto,
            page=p, size=size
        )
    return pd.DataFrame(comex_data)

In [ ]:
def calc_total_value(data):
    return data['VL_FOB'].sum()

In [ ]:
def plot_qtd_movimentada(data):
    plt.figure(figsize=(15, 10))
    plt.bar([*map(lambda x: months[x], data.keys())], data.values)
    plt.xticks(rotation=45)
    plt.title('Quantidade Movimentada')
    plt.show()

In [ ]:
def plot_util_via(data):
    plt.figure(figsize=(10, 10))
    values = data.values
    percentages = 100 * (values / values.sum())
    patches, texts = plt.pie(
        values,
        startangle=90,
        explode=(0.01,) * data.shape[0]
    )
    labels = [f'{t} - {p:.02f}%' for (t, p) in zip(data.keys(), percentages)]
    plt.legend(
        patches,
        labels,
        loc='center left',
        bbox_to_anchor=(-0.1, 1.0),
        fontsize=10
    )
    plt.tight_layout()
    plt.show()

In [ ]:
def influencia_estado(data):
    total = calc_total_value(data)
    influencia = data.groupby('SG_UF').sum()
    influencia = influencia[['VL_FOB']]
    influencia.sort_values(by='VL_FOB', inplace=True, ascending=False)
    influencia['PART'] = influencia['VL_FOB'] / total
    influencia['PART'] = influencia['PART'].apply(
        lambda x: f'{x * 100:0.2f}%'
    )
    influencia['VL_FOB'] = influencia['VL_FOB'].apply(
        lambda x: f'US$ {x:,}'.replace(',', '.')
    )
    influencia.rename(
        columns={"VL_FOB": "Valor", "PART": "Participação"},
        index=states, inplace=True
    )
    influencia.rename_axis('Estado', inplace=True)
    return influencia

In [ ]:
descricao = 'Visualização dos dados de Comércio Exterior - Governo Federal'
consulta_output = widgets.Output()


consulta_output.capture()
def consulta_on_click(ano, movimentacao, produto):
    consulta_output.clear_output()

    ano = ano or None
    movimentacao = movimentacao or None
    produto = produto or None

    data = fetch_data(ano=ano, movimentacao=movimentacao, produto=produto)
    total = calc_total_value(data)
    mes = data.groupby('MES').sum()['VL_QUANTIDADE']
    vias = data['NO_VIA'].value_counts()
    
    Tab = namedtuple('Tab', ['widget', 'title'])
    
    tab_total = widgets.Output()
    tab_qtd_mes = widgets.Output()
    tab_util_vias = widgets.Output()
    tab_influencia = widgets.Output()
    
    tabs = [
        Tab(tab_total, 'Total movimentado'),
        Tab(tab_qtd_mes, 'Quantidade movimentado Mês a Mês'),
        Tab(tab_util_vias, 'Percentual de utilização da VIA'),
        Tab(tab_influencia, 'Influência por estado no geral de importação e/ou exportação')
    ]
    
    tabs_show = widgets.Tab(children=[t.widget for t in tabs])
    _ = [tabs_show.set_title(i, t.title) for i, t in enumerate(tabs)]
    display(tabs_show)
    
    consulta_output.clear_output()
    
    with tab_total:
        display(widgets.HTML(value="<h3>Total Movimentado</h3>"))
        print(f'US$ {total:,}'.replace(',', '.'))
    with tab_qtd_mes:
        display(widgets.HTML(value="<h3>Quantidade movimentado Mês a Mês</h3>"))
        plot_qtd_movimentada(mes)
    with tab_util_vias:
        display(widgets.HTML(value="<h3>Percentual de utilização da VIA</h3>"))
        plot_util_via(vias)
    with tab_influencia:
        display(widgets.HTML(value="<h3>Influência por estado no geral de importação e/ou exportação</h3>"))
        display(influencia_estado(data))


add_accordion_tab(
    'Consulta no log de eventos',
    descricao,
    consulta_output,
    consulta_on_click,
    ano=widgets.Combobox(description='ano', options=anos, ensure_option=True),
    movimentacao=widgets.Combobox(description='movimentacao', options=movimentacoes, ensure_option=True),
    produto=widgets.Combobox(description="produto", options=produtos, ensure_option=True)
)

In [ ]:
accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(accordion_titles):
    accordion.set_title(i, title)
display(accordion)